<a href="https://colab.research.google.com/github/janithcyapa/Decentralized-HVAC-Control-System/blob/main/Basic%20Usage/energy-plus-utility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Installation**

In [5]:
!pip install -q "energy-plus-utility @ git+https://github.com/mugalan/energy-plus-utility.git@dev"

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [12]:
import os
from pyenergyplus.api import EnergyPlusAPI

# **Usage of Energy Plus Utility**

In [7]:
from eplus import prepare_colab_eplus
prepare_colab_eplus()

In [8]:
from eplus import EPlusUtil, EPlusSqlExplorer
import subprocess, json, pathlib, os
import pandas as pd
import json

## **Initialization**

### **Selection of IDF files**



For this Project I'll be using Prototype Building Models by The U.S. Department of Energy (DOE) ,  ANSI/ASHRAE/IES Standard 90.1 Prototype Building Models.


[Full Models Set](https://www.energycodes.gov/prototype-building-models)



In [2]:
# Specify the EnergyPlus Import
EPLUS_DIR = "/root/EnergyPlus-25-1-0"
OUT_DIR = "/content/eplus_out"

# Define the Simulation Model
# IDF = f"{EPLUS_DIR}/ExampleFiles/5ZoneAirCooled.idf"
IDF = f"https://raw.githubusercontent.com/janithcyapa/Decentralized-HVAC-Control-System/refs/heads/main/IDF%20Files/ASHRAE901_OfficeMedium_STD2022/ASHRAE901_OfficeMedium_STD2022_NewYork.idf"
# Select Weather Data
EPW = f"https://raw.githubusercontent.com/janithcyapa/Decentralized-HVAC-Control-System/refs/heads/main/IDF%20Files/Weather%20Files/USA_NY_New.York-J.F.Kennedy.Intl.AP.744860_TMY3.epw"


In [3]:
!rm -rf /content/eplus_out/*

In [9]:
# Create utility instance and set simulation model
util = EPlusUtil(verbose=1, out_dir = OUT_DIR)
# Reset and set model
util.delete_out_dir()
util.set_model(idf = IDF, epw = EPW,add_co2=False)
util.ensure_output_sqlite()
util.enable_runtime_logging()

FileNotFoundError: [Errno 2] No such file or directory: 'https:/raw.githubusercontent.com/janithcyapa/Decentralized-HVAC-Control-System/refs/heads/main/IDF%20Files/ASHRAE901_OfficeMedium_STD2022/ASHRAE901_OfficeMedium_STD2022_NewYork.idf'

### **Visualisation of IDF files**

In [ ]:
# Convert the model to .json
idf_path = pathlib.Path(IDF)
converter = os.path.join(EPLUS_DIR, "ConvertInputFormat")   # on Windows it's ConvertInputFormat.exe
subprocess.run([converter, str(idf_path)], check=True)
epjson_path = idf_path.with_suffix(".epJSON")

In [ ]:
# @title
def list_epjson_tables(epjson: dict):
    rows = []
    for obj_type, objs in epjson.items():
        count = len(objs) if isinstance(objs, dict) else "-"
        rows.append((obj_type, count))

    print(f"{'Object Type':45} | {'# Objects'}")
    print("-" * 60)
    for r in sorted(rows):
        print(f"{r[0]:45} | {r[1]}")

def show_object_table(epjson: dict, object_type: str):
    table = epjson.get(object_type)
    if table is None:
        print(f"{object_type} not found")
        return

    print(f"{object_type}")
    print("-" * len(object_type))
    for name, fields in table.items():
        print(f"\n{name}")
        for k, v in fields.items():
            print(f"  {k:35} : {v}")
def find_mass_flow_objects(epjson: dict):
    keywords = ["air_flow", "mass_flow", "flow_rate"]
    rows = []

    for obj_type, objs in epjson.items():
        if not isinstance(objs, dict):
            continue
        for name, fields in objs.items():
            for f in fields:
                if any(k in f.lower() for k in keywords):
                    rows.append((obj_type, name, f, fields[f]))

    print(f"{'Object Type':35} | {'Name':25} | {'Field':30} | Value")
    print("-" * 110)
    for r in rows:
        print(f"{r[0]:35} | {r[1]:25} | {r[2]:30} | {r[3]}")

In [ ]:
with open(str(epjson_path), "r", encoding="utf-8") as f:
    epjson = json.load(f)
#
# list_epjson_tables(epjson)
# show_object_table(epjson, "AirLoopHVAC")
find_mass_flow_objects(epjson)

Object Type                         | Name                      | Field                          | Value
--------------------------------------------------------------------------------------------------------------
AirLoopHVAC                         | CRAC system               | design_supply_air_flow_rate    | 8.5
AirTerminal:SingleDuct:VAV:NoReheat | Main Zone VAV Air         | constant_minimum_air_flow_fraction | 0.05
AirTerminal:SingleDuct:VAV:NoReheat | Main Zone VAV Air         | maximum_air_flow_rate          | 8.5
AirTerminal:SingleDuct:VAV:NoReheat | Main Zone VAV Air         | zone_minimum_air_flow_input_method | Constant
Coil:Cooling:DX:SingleSpeed         | Main Cooling Coil 1       | 2023_rated_evaporator_fan_power_per_volume_flow_rate | 934.4
Coil:Cooling:DX:SingleSpeed         | Main Cooling Coil 1       | rated_air_flow_rate            | 8.5
ElectricEquipment:ITE:AirCooled     | Data Center Servers       | air_flow_calculation_method    | FlowFromSystem
ElectricEquipm

## **Simulations**

### Dry run



In [ ]:
util.dry_run_min(include_ems_edd=True)

EnergyPlus Starting
EnergyPlus, Version 25.1.0-68a4a7c774, YMD=2025.12.19 06:21
Adjusting Air System Sizing
Adjusting Standard 62.1 Ventilation Sizing
Initializing Simulation
Initializing Response Factors
Calculating CTFs for "FLOOR"
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorption Factors
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Window Shading
Computing Interior Absorption Factors
Computing Interior Diffuse Solar Absorption Factors
Initializing Solar Heat Gains
Initializing Internal Heat Gains
Initializing Interior Solar Distribution
Initializing Interior Conve

0

In [ ]:
err_path = f"{OUT_DIR}/eplusout.err"
print(open(err_path, "r", errors="ignore").read()[-4000:])

Program Version,EnergyPlus, Version 25.1.0-68a4a7c774, YMD=2025.12.19 05:50,
   ** Warning ** Weather file location will be used rather than entered (IDF) Location object.
   **   ~~~   ** ..Location object=DENVER CENTENNIAL CO USA WMO=724666
   **   ~~~   ** ..Weather File Location=San Francisco Intl Ap CA USA TMY3 WMO#=724940
   **   ~~~   ** ..due to location differences, Latitude difference=[2.12] degrees, Longitude difference=[17.22] degrees.
   **   ~~~   ** ..Time Zone difference=[1.0] hour(s), Elevation difference=[99.89] percent, [1791.00] meters.
   ** Warning ** SetUpDesignDay: Entered DesignDay Barometric Pressure=81560 differs by more than 10% from Standard Barometric Pressure=101301.
   **   ~~~   ** ...occurs in DesignDay=DENVER CENTENNIAL ANN HTG 99.6% CONDNS DB, Standard Pressure (based on elevation) will be used.
   ** Warning ** SetUpDesignDay: Entered DesignDay Barometric Pressure=81560 differs by more than 10% from Standard Barometric Pressure=101301.
   **   ~~~  

In [ ]:
util.list_zone_names()

['Main Zone']

In [ ]:
util.list_available_variables()

,Kind,VariableName,KeyValue,Units
0,Outputvariable,Site Outdoor Air Drybulb Temperature,ENVIRONMENT,C
1,Outputvariable,Site Outdoor Air Dewpoint Temperature,ENVIRONMENT,C
2,Outputvariable,Site Outdoor Air Wetbulb Temperature,ENVIRONMENT,C
3,Outputvariable,Site Outdoor Air Humidity Ratio,ENVIRONMENT,kgWater/kgDryAir
4,Outputvariable,Site Outdoor Air Relative Humidity,ENVIRONMENT,%
...,...,...,...,...
142,Outputvariable,System Node Setpoint Temperature,MAIN ZONE ATU IN NODE,C
143,Outputvariable,System Node Standard Density Volume Flow Rate,MAIN ZONE ATU IN NODE,m3/s
144,Outputvariable,Environmental Impact Total N2O Emissions Carbo...,SITE,kg
145,Outputvariable,Environmental Impact Total CH4 Emissions Carbo...,SITE,kg


In [ ]:
util.list_available_actuators()


,Kind,ComponentType,ControlType,ActuatorKey,Units
0,Actuator,Schedule:Compact,Schedule Value,OCCUPY-1,[ ]
1,Actuator,Schedule:Compact,Schedule Value,LIGHTS-1,[ ]
2,Actuator,Schedule:Compact,Schedule Value,EQUIP-1,[ ]
3,Actuator,Schedule:Compact,Schedule Value,INFIL-SCH,[ ]
4,Actuator,Schedule:Compact,Schedule Value,ACTSCHD,[ ]
...,...,...,...,...,...
1209,Actuator,Outdoor Air System Node,Drybulb Temperature,CENTRAL CHILLER CONDENSER INLET NODE,[C]
1210,Actuator,Outdoor Air System Node,Wetbulb Temperature,CENTRAL CHILLER CONDENSER INLET NODE,[C]
1211,Actuator,Outdoor Air System Node,Wind Speed,CENTRAL CHILLER CONDENSER INLET NODE,[m/s]
1212,Actuator,Outdoor Air System Node,Wind Direction,CENTRAL CHILLER CONDENSER INLET NODE,[degree]


In [ ]:
util.list_available_meters()

,Kind,MeterName,Units


### Config Simulatin

In [ ]:
def pid_zone_temp_to_mdot(self, *args, **kwargs):
    # ----- pick zone name safely (wrapper may inject it) -----
    zone_from_args = args[0] if len(args) >= 1 and isinstance(args[0], str) else None
    zone_name = kwargs.pop("zone_name", None) or kwargs.pop("zone_key", None) or zone_from_args or "Main Zone"

    # ----- params -----
    actuator_node = kwargs.pop("actuator_node", "Main Zone ATU In Node")
    T_set        = float(kwargs.pop("T_set", 25.0))
    Kp           = float(kwargs.pop("Kp", 0.20))
    Ki           = float(kwargs.pop("Ki", 0.002))
    Kd           = float(kwargs.pop("Kd", 0.0))
    mdot_min     = float(kwargs.pop("mdot_min", 0.05))
    mdot_max     = float(kwargs.pop("mdot_max", 1.50))
    log_every    = int(kwargs.pop("log_every_steps", 0))

    # ----- robust variable read (supports different util APIs) -----
    def _read_var(var_name, key):
        # Try the most likely method names
        for mname in ("var_get", "get_var", "read_variable", "get_output_variable"):
            if hasattr(self, mname):
                return getattr(self, mname)(var_name, key)
        # As a fallback: if util exposes a logger reader, sometimes it's named tick_get_variable
        if hasattr(self, "tick_get_variable"):
            return self.tick_get_variable(name=var_name, key=key)
        raise AttributeError("No variable-read method found on util (tried var_get/get_var/read_variable/get_output_variable/tick_get_variable).")

    Tz = _read_var("Zone Mean Air Temperature", zone_name)
    if Tz is None:
        return

    # ----- persistent PID state -----
    if not hasattr(self, "_pid_state"):
        self._pid_state = {"I": 0.0, "e_prev": None, "step": 0}
    st = self._pid_state
    st["step"] += 1

    dt = float(getattr(self, "dt_seconds", 600.0))  # fallback 10 min

    # error: positive if too hot -> increase flow
    e = float(Tz) - T_set

    # integral
    st["I"] += e * dt

    # derivative
    if st["e_prev"] is None:
        de_dt = 0.0
    else:
        de_dt = (e - st["e_prev"]) / dt
    st["e_prev"] = e

    # PID -> absolute mdot
    mdot_bias = 0.5 * (mdot_min + mdot_max)
    mdot = mdot_bias + Kp * e + Ki * st["I"] + Kd * de_dt
    mdot = max(mdot_min, min(mdot_max, mdot))

    # ----- robust actuator write (supports different util APIs) -----
    ok = None
    if hasattr(self, "set_actuator"):
        ok = self.set_actuator(
            component_type="System Node Setpoint",
            control_type="Mass Flow Rate Setpoint",
            actuator_key=actuator_node,
            value=mdot,
        )
    elif hasattr(self, "tick_set_actuator"):
        ok = self.tick_set_actuator(
            component_type="System Node Setpoint",
            control_type="Mass Flow Rate Setpoint",
            actuator_key=actuator_node,
            value=mdot,
            when="success",
            read_back=True,
            precision=4,
        )
    else:
        raise AttributeError("No actuator-write method found on util (tried set_actuator/tick_set_actuator).")

    if log_every and (st["step"] % log_every == 0):
        print(f"[PID] zone={zone_name} Tz={float(Tz):.2f}C e={e:.2f} mdot={mdot:.3f} ok={ok}")


In [ ]:
util.__class__.pid_zone_temp_to_mdot = pid_zone_temp_to_mdot


In [ ]:
specs = [
    # --- Zone state + people ---
    {"name": "Zone Mean Air Temperature",                "key": "*",            "freq": "TimeStep"},
    {"name": "Zone Mean Air Dewpoint Temperature",       "key": "*",            "freq": "TimeStep"},
    {"name": "Zone Air Relative Humidity",               "key": "*",            "freq": "TimeStep"},
    {"name": "Zone Mean Air Humidity Ratio",             "key": "*",            "freq": "TimeStep"},
    {"name": "Zone People Occupant Count",               "key": "*",            "freq": "TimeStep"},

    # --- CO₂ & OA into zones ---
    {"name": "Zone Air CO2 Concentration",               "key": "*",            "freq": "TimeStep"},

    # --- Site weather (Environment key) ---
    {"name": "Site Outdoor Air Drybulb Temperature",     "key": "Environment",  "freq": "TimeStep"},
    {"name": "Site Outdoor Air Wetbulb Temperature",     "key": "Environment",  "freq": "TimeStep"},
    {"name": "Site Outdoor Air Dewpoint Temperature",     "key": "Environment",  "freq": "TimeStep"},
    {"name": "Site Outdoor Air Relative Humidity",     "key": "Environment",  "freq": "TimeStep"},
    {"name": "Site Outdoor Air Humidity Ratio",     "key": "Environment",  "freq": "TimeStep"},
    {"name": "Site Outdoor Air Barometric Pressure",        "key": "Environment", "freq": "TimeStep"},
    {"name": "Site Outdoor Air CO2 Concentration",                          "key": "Environment",  "freq": "TimeStep"},

    {"name": "System Node Temperature",               "key": "*",            "freq": "TimeStep"},
    {"name": "System Node Mass Flow Rate",               "key": "*",            "freq": "TimeStep"},
    {"name": "System Node Humidity Ratio",               "key": "*",            "freq": "TimeStep"},
    {"name": "System Node CO2 Concentration",               "key": "*",            "freq": "TimeStep"},
]

# 1) Ensure the Output:Variable objects exist (dedup-aware)
util.ensure_output_variables(specs, activate=True)


# 2) Ensure the meter(s) you want are reported
output_meters = [
    "Electricity:Facility",
    "ElectricityPurchased:Facility",
    "ElectricitySurplusSold:Facility",
    "Cooling:EnergyTransfer",
    # NOTE: The two below are model-specific. Keep them only if they exist in your IDF.
    # "InteriorLights:Electricity:Zone:SPACE5-1",
    # "Cooling:EnergyTransfer:Zone:SPACE1-1",
]
util.ensure_output_meters(output_meters, freq="TimeStep")


#3) Register callbacks
util.register_handlers(
    "after_hvac",
    [{
        "method_name": "tick_log_variable",
        "kwargs": {
            "name": "Zone Mean Air Temperature",
            "key": "Main Zone",          # <-- change if your zone name differs
            "when": "always",
            "precision": 2,
            "include_timestamp": True,
            "allow_warmup": False
        }
    }],
    clear=False,
    run_during_warmup=False
)


# util.register_handlers(
#     "before_hvac",
#     [{
#         "method_name": "pid_zone_temp_to_mdot",
#         "kwargs": {
#             # DO NOT pass zone_name / zone_key
#             "actuator_node": "Main Zone ATU In Node",  # may need to change to a valid actuator key
#             "T_set": 25.0,
#             "Kp": 0.20,
#             "Ki": 0.002,
#             "Kd": 0.0,
#             "mdot_min": 0.05,
#             "mdot_max": 1.50,
#             "log_every_steps": 30
#         }
#     }],
#     clear=True,
#     run_during_warmup=False
# )


util.register_handlers(
    "before_hvac",
    [{
        "method_name": "tick_set_actuator",
        "kwargs": {
            "component_type": "Schedule:Compact",      # sometimes "Schedule:Constant" etc.
            "control_type":   "Schedule Value",
            "actuator_key":   "Cooling Return Air Setpoint Schedule",
            "value":          25.0,
            "when":           "success",
            "read_back":      True,
            "precision":      2
        }
    }],
    clear=True,
    run_during_warmup=False
)

rc=util.run_annual()

EnergyPlus Starting
EnergyPlus, Version 25.1.0-68a4a7c774, YMD=2025.12.19 06:23
Adjusting Air System Sizing
Adjusting Standard 62.1 Ventilation Sizing
Initializing Simulation
Initializing Response Factors
Calculating CTFs for "FLOOR"
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorption Factors
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Window Shading
Computing Interior Absorption Factors
Computing Interior Diffuse Solar Absorption Factors
Initializing Solar Heat Gains
Initializing Internal Heat Gains
Initializing Interior Solar Distribution
Initializing Interior Conve

In [ ]:
util.list_available_variables()

,Kind,VariableName,KeyValue,Units
0,Outputvariable,Site Outdoor Air Drybulb Temperature,ENVIRONMENT,C
1,Outputvariable,Site Outdoor Air Drybulb Temperature,ENVIRONMENT,C
2,Outputvariable,Site Outdoor Air Dewpoint Temperature,ENVIRONMENT,C
3,Outputvariable,Site Outdoor Air Dewpoint Temperature,ENVIRONMENT,C
4,Outputvariable,Site Outdoor Air Wetbulb Temperature,ENVIRONMENT,C
...,...,...,...,...
190,Outputvariable,System Node Setpoint Temperature,MAIN ZONE ATU IN NODE,C
191,Outputvariable,System Node Standard Density Volume Flow Rate,MAIN ZONE ATU IN NODE,m3/s
192,Outputvariable,Environmental Impact Total N2O Emissions Carbo...,SITE,kg
193,Outputvariable,Environmental Impact Total CH4 Emissions Carbo...,SITE,kg


In [ ]:
xp = EPlusSqlExplorer(f"{OUT_DIR}/eplusout.sql")
xp.list_tables()
df = xp.list_sql_variables(name="System Node Temperature")
df[['KeyValue','n_rows']].head(20)

,KeyValue,n_rows
0,MAIN COOLING COIL 1 OUTLET NODE,288
1,MAIN ZONE ATU IN NODE,288
2,MAIN ZONE INLET NODE,288
3,MAIN ZONE NODE,288
4,MAIN ZONE OUTLET NODE,288
5,SUPPLY INLET NODE,288
6,SUPPLY OUTLET NODE,288
7,ZONE EQUIPMENT INLET NODE,288
8,ZONE EQUIPMENT OUTLET NODE,288
9,MAIN COOLING COIL 1 OUTLET NODE,48


In [ ]:
# Drybulb (auto-picks top zone keys if keys=None)
fig1=util.plot_sql_zone_variable(
    "Zone Mean Air Temperature",
    keys=["*"],                          # auto-pick a few zones with data
    reporting_freq=("TimeStep",),       # match how you logged
    resample="1h",
    title="Zone Mean Air Temperature"
)

# Humidity ratio
fig2=util.plot_sql_zone_variable(
    "Zone Mean Air Humidity Ratio",
    keys=["*"],
    reporting_freq=("TimeStep",),
    resample="1h",
    title="Zone Mean Air Humidity Ratio"
)

# CO2 concentration
# fig3=util.plot_sql_zone_variable(
#     "Zone Air CO2 Concentration",
#     keys={"*"},
#     reporting_freq=("TimeStep",),
#     resample="1h",
#     title="Zone Air CO2 Concentration"
# )

# **Use EnergyPlus Python API**

In [1]:
# --- CELL 1: Install EnergyPlus 22.1.0 ---
# Download EnergyPlus 22.1.0 (Matches your IDF file)
!wget https://github.com/NREL/EnergyPlus/releases/download/v22.1.0/EnergyPlus-22.1.0-ed759b17ee-Linux-Ubuntu20.04-x86_64.sh

# Make executable
!chmod +x EnergyPlus-22.1.0-ed759b17ee-Linux-Ubuntu20.04-x86_64.sh

# Install to /usr/local/EnergyPlus-22-1-0
!sudo ./EnergyPlus-22.1.0-ed759b17ee-Linux-Ubuntu20.04-x86_64.sh --skip-license --prefix=/usr/local --exclude-subdir

# Clean up
!rm EnergyPlus-22.1.0-ed759b17ee-Linux-Ubuntu20.04-x86_64.sh

--2026-01-13 04:50:04--  https://github.com/NREL/EnergyPlus/releases/download/v22.1.0/EnergyPlus-22.1.0-ed759b17ee-Linux-Ubuntu20.04-x86_64.sh
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/14620185/fd1f35c5-e266-4251-a6ab-d6f1283d5e84?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-01-13T05%3A34%3A54Z&rscd=attachment%3B+filename%3DEnergyPlus-22.1.0-ed759b17ee-Linux-Ubuntu20.04-x86_64.sh&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2026-01-13T04%3A34%3A13Z&ske=2026-01-13T05%3A34%3A54Z&sks=b&skv=2018-11-09&sig=9kGFir9xwJSBx%2BUUQw9Lrhc6gATPFq7k%2F%2B7m3kBgZz8%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cC

In [3]:
eplus_install_path = "/usr/local/EnergyPlus-22-1-0"
sys.path.insert(0, eplus_install_path)

In [4]:
import sys
import os
from pyenergyplus.api import EnergyPlusAPI
import pandas as pd
import matplotlib.pyplot as plt

In [18]:
# Import IDF and Weather Files
# Define the RAW GitHub URLs (Ensure they are the 'raw' versions)
idf_url = "https://raw.githubusercontent.com/janithcyapa/Decentralized-HVAC-Control-System/refs/heads/main/IDF%20Files/ASHRAE901_OfficeMedium_STD2022/ASHRAE901_OfficeMedium_STD2022_NewYork.idf"
epw_url = "https://raw.githubusercontent.com/janithcyapa/Decentralized-HVAC-Control-System/refs/heads/main/IDF%20Files/Weather%20Files/USA_NY_New.York-J.F.Kennedy.Intl.AP.744860_TMY3.epw"

# Define local filenames for Colab
idf_local = "medium_office.idf"
epw_local = "weather.epw"

# Download the files using system commands
os.system(f"wget -O {idf_local} {idf_url}")
os.system(f"wget -O {epw_local} {epw_url}")

print("Files downloaded successfully:")
print(f"IDF Model: {os.path.abspath(idf_local)}")
print(f"Weather:   {os.path.abspath(epw_local)}")

Files downloaded successfully:
IDF Model: /content/medium_office.idf
Weather:   /content/weather.epw


In [19]:
idf_path = "medium_office.idf"
epw_path = "weather.epw"
output_dir = "simulation_output"

# 4. Initialize and Run
api = EnergyPlusAPI()
state = api.state_manager.new_state()

In [21]:
# This prints the startup banner and standard messages to the console
api.runtime.set_console_output_status(state, True)

# This will print "Simulation Progress: X%" as it runs
api.runtime.callback_progress(state, progress_handler)

print("--- Starting EnergyPlus Simulation ---")

result = api.runtime.run_energyplus(
    state,
    [
        '-w', epw_path,        # Weather file
        '-d', output_dir,      # Output directory
        idf_path               # Input file
    ]
)

print("--- Simulation Finished ---")

if result == 0:
    print("Status: SUCCESS")
else:
    print("Status: FAILED")

    # Helper: Read the error log immediately if it fails
    err_file = os.path.join(output_dir, 'eplusout.err')
    if os.path.exists(err_file):
        print("\n--- Error Log Content ---")
        with open(err_file, 'r') as f:
            print(f.read())

--- Starting EnergyPlus Simulation ---
--- Simulation Finished ---
Status: FAILED

--- Error Log Content ---
Program Version,EnergyPlus, Version 22.1.0-ed759b17ee, YMD=2026.01.13 05:00,
   ** Severe  ** Line: 11394 Index: 13 - "Output:Format" is not a valid Object Type.
   **  Fatal  ** Errors occurred on processing input file. Preceding condition(s) cause termination.
   ...Summary of Errors that led to program termination:
   ..... Reference severe error count=1
   ..... Last severe error=Line: 11394 Index: 13 - "Output:Format" is not a valid Object Type.
   ************* Warning:  Node connection errors not checked - most system input has not been read (see previous warning).
   ************* Fatal error -- final processing.  Program exited before simulations began.  See previous error messages.
   ************* EnergyPlus Warmup Error Summary. During Warmup: 0 Warning; 0 Severe Errors.
   ************* EnergyPlus Sizing Error Summary. During Sizing: 0 Warning; 0 Severe Errors.
   *

In [12]:
# 1. Define the path to the results
results_file = 'simulation_output/eplusout.csv'

# Check if file exists first
if not os.path.exists(results_file):
    print(f"Error: Could not find {results_file}.")
    print("Make sure your simulation ran successfully (Step 3) before running this.")
else:
    # 2. Load the data into a DataFrame
    # EnergyPlus CSVs have headers in the first row
    df = pd.read_csv(results_file)

    # 3. Clean up the Date/Time column for plotting
    # (EnergyPlus uses a non-standard date format like " 1/ 1  00:15:00")
    # For a quick look, we will just use the Simulation Step Index
    df['Step_Index'] = df.index

    # 4. Find columns related to "Zone Mean Air Temperature"
    # This automatically finds the correct column names for your specific building
    temp_cols = [col for col in df.columns if 'Zone Mean Air Temperature' in col]

    if not temp_cols:
        print("Warning: No 'Zone Mean Air Temperature' columns found in the CSV.")
        print("You may need to add an 'Output:Variable' object to your IDF file.")
    else:
        # 5. Plot the data
        plt.figure(figsize=(15, 6))

        # Plot the first 5 zones found (to keep the graph readable)
        for col in temp_cols[:5]:
            plt.plot(df['Step_Index'], df[col], label=col)

        plt.title('Zone Temperatures Over Time')
        plt.xlabel('Simulation Time Steps')
        plt.ylabel('Temperature [C]')
        plt.legend(loc='best', fontsize='small')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

        # Print the first few rows so you can see the raw data
        print("\n--- First 5 rows of Data ---")
        print(df[temp_cols].head())


Error: Could not find simulation_output/eplusout.csv.
Make sure your simulation ran successfully (Step 3) before running this.


# Utils

In [9]:
def progress_handler(percent):
    # This function is called by EnergyPlus every time progress updates
    print(f"Simulation Progress: {percent}%")